In [1]:
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)

In [40]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = pd.DataFrame(teams.get_teams())
#print('Number of teams fetched: {}'.format(len(nba_teams)))
#nba_teams

#create a list of all ids
team_ids = nba_teams['id'].tolist()
print(team_ids)

[1610612737, 1610612738, 1610612739, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]


In [47]:
from nba_api.stats.endpoints import leaguegamefinder
#get all games for team_ids
games = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_ids).get_data_frames()



In [37]:
from nba_api.stats.endpoints import leaguegamefinder

#select dictionary of just NBA teams
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_ids)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()
games

#games object has NBA games from the current season to 1985

[     SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
 0        22021  1610612737               ATL  Atlanta Hawks  0022101085   
 1        22021  1610612737               ATL  Atlanta Hawks  0022101081   
 2        22021  1610612737               ATL  Atlanta Hawks  0022101064   
 3        22021  1610612737               ATL  Atlanta Hawks  0022101047   
 4        22021  1610612737               ATL  Atlanta Hawks  0022101033   
 ...        ...         ...               ...            ...         ...   
 3483     21983  1610612737               ATL  Atlanta Hawks  0028300058   
 3484     21983  1610612737               ATL  Atlanta Hawks  0028300041   
 3485     21983  1610612737               ATL  Atlanta Hawks  0028300027   
 3486     21983  1610612737               ATL  Atlanta Hawks  0028300014   
 3487     21983  1610612737               ATL  Atlanta Hawks  0028300005   
 
        GAME_DATE      MATCHUP WL  MIN  PTS  FGM  FGA  FG_PCT  FG3M  FG3A  \
 0     20

In [14]:
list(games)

['SEASON_ID',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PLUS_MINUS']

In [34]:

# Subset Season by year
# Subset the games to when the last 4 digits of SEASON_ID is in the list s.
s =['2017','2018','2019','2020','2021']
season_2017to2021 = games[games.SEASON_ID.str[-4:].isin(s)]  # subset to 2017-2021
#season_2017to2021

AttributeError: 'list' object has no attribute 'SEASON_ID'

In [3]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Mavs, which contains their team ID
Mavs = [team for team in nba_teams if team['abbreviation'] == 'DAL'][0]
Mavs_id = Mavs['id']

In [4]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Mavs were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=Mavs_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612742,DAL,Dallas Mavericks,0022101092,2022-03-23,DAL vs. HOU,W,240,110,42,86,0.488,8,32,0.250,18,22,0.818,8,34,42,24,9,1,12,24,19.0
1,22021,1610612742,DAL,Dallas Mavericks,0022101079,2022-03-21,DAL vs. MIN,W,241,110,40,83,0.482,14,46,0.304,16,21,0.762,7,40,47,27,5,3,11,21,2.0
2,22021,1610612742,DAL,Dallas Mavericks,0022101059,2022-03-19,DAL @ CHA,L,241,108,39,89,0.438,14,38,0.368,16,20,0.800,12,32,44,18,4,4,13,26,-21.0
3,22021,1610612742,DAL,Dallas Mavericks,0022101046,2022-03-18,DAL @ PHI,L,241,101,37,80,0.463,12,32,0.375,15,21,0.714,11,29,40,21,4,3,12,17,-10.0
4,22021,1610612742,DAL,Dallas Mavericks,0022101036,2022-03-16,DAL @ BKN,W,239,113,41,83,0.494,12,34,0.353,19,19,1.000,9,28,37,21,5,2,10,17,2.0


In [6]:
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()

In [7]:
games

[      SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME  \
 0         22021  1612709932               MCC      Motor City Cruise   
 1         22021  1612709910               FWN    Fort Wayne Mad Ants   
 2         22021  1610612766               CHA      Charlotte Hornets   
 3         22021  1612709922               GBO       Greensboro Swarm   
 4         22021  1612709925               LAK         Lakeland Magic   
 ...         ...         ...               ...                    ...   
 29995     22015  1610612759               SAS      San Antonio Spurs   
 29996     22015  1612709905               LAD  Los Angeles D-Fenders   
 29997     22015  1612709920               RAP            Raptors 905   
 29998     22015  1612709918               TEX          Texas Legends   
 29999     22015  1610612739               CLE    Cleveland Cavaliers   
 
           GAME_ID   GAME_DATE      MATCHUP WL  MIN  PTS  FGM  FGA  FG_PCT  \
 0      2022100445  2022-03-24    MCC @ FWN 